In [1]:
import sys  # 导入sys模块
import time

import numpy as np

sys.setrecursionlimit(3000)
import pandas as pd
import matplotlib.pyplot as plt

## 说明
- 生成 9*9*9 矩阵 依次对应 1～9 相位（NumStage），row，line/clo；
- 9块区域用group代表
- 确定/可能/不可能 依次用 10/1/0 来代表

## 生成数独 table

In [2]:
def creat_new_table(start_list):
    #根据录入列表生成table
    start_np=np.array(start_list1).reshape((9,9))
    the_np=np.ones((9,9,9))*start_np
    the_np=np.where(the_np==0,np.ones((9,9,9)),0)
    np_ls=[]
    num=1
    for i in the_np:
        np_ls.append(np.where(start_np==num,10,i))
        num=num+1
    the_table=np.concatenate(np_ls,axis=0).reshape((9,9,9)).astype(np.int32)
    
    return the_table

## 更新矩阵

#### 生成group_np

In [3]:
def creat_group_np():
    # 生成检查group 用 np
    group_div=[
        [0,3,0,3],
        [0,3,3,6],
        [0,3,6,9],
        [3,6,0,3],
        [3,6,3,6],
        [3,6,6,9],
        [6,9,0,3],
        [6,9,3,6],
        [6,9,6,9]  
        ]
    np_ls=[]
    for nums in group_div:
        group_np=np.zeros((9,9))
        group_np[nums[0]:nums[1],nums[2]:nums[3]]=1
        np_ls.append(group_np)
    the_table=np.concatenate(np_ls,axis=0).reshape((9,9,9)).astype(np.int32)

    return the_table

### num_stage 确定后，将相关数1 to 0

#### 将group 1 to 0 

In [4]:
def group_1to0(new_table):
    group_div=[
        [0,3,0,3],
        [0,3,3,6],
        [0,3,6,9],
        [3,6,0,3],
        [3,6,3,6],
        [3,6,6,9],
        [6,9,0,3],
        [6,9,3,6],
        [6,9,6,9]  
        ]
    for num_stage in range(9):
        the_where=np.array(np.where((new_table[num_stage]*creat_group_np())==10)).T
        for i in the_where:
            # 数组i 的0,1,2 依次为 group，row,col

            the_div=group_div[i[0]] #提取group 的区间
            new_table[num_stage,the_div[0]:the_div[1],the_div[2]:the_div[3]]=0 #将group区间都更新为0

            new_table[num_stage][i[1],i[2]]=10#将原10位置更新为10
    return new_table

In [5]:
def table_1to0(the_table):
    # 将10_block相关都更新为0
    the_where=np.array(np.where(the_table==10)).T
    for i in the_where:
        the_table[i[0],i[1]]=0
        the_table[i[0],:,i[2]]=0
        the_table[:,i[1],i[2]]=0
        the_table[i[0],i[1],i[2]]=10
    the_table=group_1to0(the_table)
    return the_table

### 确定num_stage 1改为10

In [6]:
def the_one_0(the_table):
    #将num_stage 为1替换为10
    the_one=np.sum(the_table,axis=0)
    if 1 in the_one:
        the_where=np.array(np.where(the_one==1)).T
        for i in the_where:
            the_table[:,i[0],i[1]]=np.where(the_table[:,i[0],i[1]]==1,10,the_table[:,i[0],i[1]])
    return the_table

In [7]:
def the_one_1(the_table):
    #将col_stage 为1替换为10
    the_one=np.sum(the_table,axis=1)
    if 1 in the_one:
        the_where=np.array(np.where(the_one==1)).T
        for i in the_where:
            the_table[i[0],:,i[1]]=np.where(the_table[i[0],:,i[1]]==1,10,the_table[i[0],:,i[1]])
    return the_table

In [8]:
def the_one_2(the_table):
    #将line_stage 为1替换为10
    the_one=np.sum(the_table,axis=2)
    if 1 in the_one:
        the_where=np.array(np.where(the_one==1)).T
        for i in the_where:
            the_table[i[0],i[1],:]=np.where(the_table[i[0],i[1],:]==1,10,the_table[i[0],i[1],:])
    return the_table

### 检测是否有独立带1的block

In [9]:
def check_one_in(the_table):
    # 判断是否有1需要替换为10的项目
    if 1 in np.sum(the_table,axis=0):
        return True
    elif 1 in np.sum(the_table,axis=1):
        return True
    elif 1 in np.sum(the_table,axis=2):
        return True
    else:
        return False

### 检查是否互斥 twin block

In [10]:
def rebulit_twin_col(the_table,the_little,the_big):
    
    little_row,little_col=the_little[0],the_little[1]
    big_row,big_col=the_big[0],the_big[1]
    if (the_table[:,little_row,little_col]==the_table[:,big_row,big_col]).all():
        #判断是否是twins
        
        #提取twin num_stage
        i=np.where((the_table[:,little_row,little_col])==1)
        twin_0,twin_1=i[0][0],i[0][1]
        #将同row都剔除 twins
        the_table[twin_0,:,little_col]=0
        the_table[twin_1,:,little_col]=0

        #将twinblock 恢复
        the_table[twin_0,little_row,little_col]=1
        the_table[twin_1,little_row,little_col]=1
        the_table[twin_0,big_row,big_col]=1
        the_table[twin_1,big_row,big_col]=1
        
    return the_table

In [11]:
def rebulit_twin_row(the_table,the_little,the_big):
    
    little_row,little_col=the_little[0],the_little[1]
    big_row,big_col=the_big[0],the_big[1]
    if (the_table[:,little_row,little_col]==the_table[:,big_row,big_col]).all():
        #判断是否是twins
       
        #提取twin num_stage
        i=np.where((the_table[:,little_row,little_col])==1)
        twin_0,twin_1=i[0][0],i[0][1]
        #将同row都剔除 twins
        the_table[twin_0,little_row,:]=0
        the_table[twin_1,little_row,:]=0

        #将twinblock 恢复
        the_table[twin_0,little_row,little_col]=1
        the_table[twin_1,little_row,little_col]=1
        the_table[twin_0,big_row,big_col]=1
        the_table[twin_1,big_row,big_col]=1
        
    return the_table

In [12]:
def check_the_twin(the_table):
    #检查是否有互锁twin
    the_two=np.sum(the_table,axis=0)
    the_two_where=np.array(np.where(the_two==2)).T
    i=0
    while i<len(the_two_where):
        the_little=the_two_where[i]
       
        j=i+1
        while j<len(the_two_where):
            the_big=the_two_where[j]
            if the_little[0]==the_big[0]:
                #检查两个是否相同
                the_table=rebulit_twin_row(the_table,the_little,the_big)

                j=j+1
            elif the_little[1]==the_big[1]:
                #检查两个是否相同
                the_table=rebulit_twin_col(the_table,the_little,the_big)
                j=j+1
            else:
                j=j+1
        i=i+1
    return the_table

## 循环检测更新 the_table

In [13]:
def time_used(fun):
    def wrapper(*args, **kwargs):
        time_start=time.time()
        ans=fun(*args, **kwargs)        
        time_use=(time.time()-time_start)*1000       
        print("\n\n time used :%.2f ms\n"%time_use)
        return ans
    return wrapper

In [14]:
def cycle_check(the_table):
    # 循环检测其中需要更新项目
    the_table=table_1to0(the_table)
    while check_one_in(the_table):
        the_table=the_one_0(the_table)
        the_table=the_one_1(the_table)
        the_table=the_one_2(the_table)
        the_table=table_1to0(the_table)
    return the_table

In [15]:
@time_used
def find_the_answer(start_list):
   
    the_table=creat_new_table(start_list)
    i=0
    while i<3:
        the_table=cycle_check(the_table)
        the_table=check_the_twin(the_table)
        i=i+1
    return the_table

## 展示结果

In [16]:
def show_table(the_table):
    #show the table maybe
    the_add=(the_table.transpose((1,2,0))).reshape((81,9)) # 对table转置
    the_add=np.where(the_add==10,1,the_add)*np.linspace(1,9,num=9,dtype=int)
    
    the_ls=[]
    the_add=the_add.astype(int)
    for i in the_add:
    #     print(i)
        ls=''
        for j in i:
            if j!=0:
                j=j.astype(str)
                ls=ls+j
        the_ls.append(ls)
        
    new_ls=[]
    for i in range(9):
        new_ls.append(the_ls[i*9:i*9+9])
    return pd.DataFrame(new_ls)

## 测试

In [17]:
# 开局信息1
start_list1=[
    0,8,0,0,0,7,0,0,6,
    0,3,9,2,0,8,0,0,0,
    4,0,0,1,3,0,0,0,0,
    9,0,5,0,0,0,0,0,4,
    0,0,3,0,0,0,0,0,0,
    6,2,0,0,0,5,0,0,0,
    8,0,1,0,0,6,2,0,0,
    0,0,0,5,0,0,0,0,0,
    0,6,0,3,0,0,8,0,0
]
# 开局信息_2
start_list2=[
    5,0,2,0,0,0,0,8,0,
    0,0,1,0,0,0,0,0,0,
    0,0,0,4,0,3,0,0,0,
    0,4,0,0,0,0,0,0,0,
    0,0,0,0,0,6,7,0,9,
    0,0,8,0,5,0,0,0,0,
    0,0,0,0,2,1,0,0,0,
    0,9,0,0,0,0,0,0,3,
    0,7,0,0,0,0,6,0,0
]
# 开局信息_3
start_list3=[
    0,0,0,0,5,8,0,0,0,
    7,6,0,0,0,2,3,0,4,
    0,0,0,3,4,0,8,2,0,
    0,7,0,0,0,0,0,0,1,
    0,0,0,0,0,0,6,4,0,
    1,3,2,4,6,0,9,0,0,
    3,0,0,7,9,0,2,0,5,
    0,0,7,0,0,4,1,0,0,
    5,9,0,0,0,6,0,0,8,
]

In [18]:
# 开局信息_4
start_list4=[
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
  ]
# 开局信息_4
start_list4=[
    0,4,6,9,0,3,0,0,0,
    0,0,3,0,5,0,0,6,0,
    9,0,0,0,0,2,0,0,3,
    0,0,5,0,0,6,0,0,0,
    8,0,0,0,0,0,0,1,0,
    0,1,0,7,8,0,2,0,0,
    0,0,0,0,0,0,0,5,0,
    0,8,1,3,0,0,0,0,7,
    0,0,0,8,0,0,1,0,4,
  ]
# 开局信息_5
start_list5=[
    8,0,0,0,0,0,0,0,0,
    0,0,3,6,0,0,0,0,0,
    0,7,0,0,9,0,2,0,0,
    0,5,0,0,0,7,0,0,0,
    0,0,0,0,4,5,7,0,0,
    0,0,0,1,0,0,0,3,0,
    0,0,1,0,0,0,0,6,8,
    0,0,8,5,0,0,0,1,0,
    0,9,0,0,0,0,4,0,0,
  ]
# 开局信息_6
start_list6=[
    8,0,0,0,0,0,0,0,0,
    0,0,0,0,0,5,7,0,0,
    0,0,0,0,3,0,0,9,1,
    5,0,0,0,0,3,0,0,2,
    0,3,0,6,0,8,0,0,0,
    0,0,0,0,9,0,0,0,8,
    0,2,7,5,0,0,9,1,0,
    0,1,0,0,0,4,0,0,0,
    0,0,3,1,0,0,4,5,0,
  ]

In [19]:
# 开局信息_0
start_list0=[
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,9,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
  ]

In [20]:
# 开局信息_7
start_list7=[
    0,0,1,2,0,3,4,0,0,
    0,0,0,6,0,7,0,0,0,
    5,0,0,0,0,0,0,0,3,
    3,7,0,0,0,0,0,8,1,
    0,0,0,0,0,0,0,0,0,
    6,2,0,0,9,0,0,3,7,
    1,0,0,0,0,0,0,0,8,
    0,0,0,8,0,5,0,0,0,
    0,0,6,4,0,2,5,0,0,
  ]

In [21]:
show_table(find_the_answer(start_list7))



 time used :9.00 ms



,0,1,2,3,4,5,6,7,8
0,1,8,2,4,5,7,9,3,6
1,5,3,9,2,6,8,4,7,1
2,4,7,6,1,3,9,5,2,8
3,9,1,5,6,2,3,7,8,4
4,7,4,3,8,9,1,6,5,2
5,6,2,8,7,4,5,3,1,9
6,8,5,1,9,7,6,2,4,3
7,3,9,4,5,8,2,1,6,7
8,2,6,7,3,1,4,8,9,5


In [22]:
the_ls=[start_list1,start_list2,start_list3,start_list4,start_list5,start_list6,start_list7]
time_ls=[]
for i in the_ls:
    print(show_table(find_the_answer(i)))



 time used :8.93 ms

   0  1  2  3  4  5  6  7  8
0  1  8  2  4  5  7  9  3  6
1  5  3  9  2  6  8  4  7  1
2  4  7  6  1  3  9  5  2  8
3  9  1  5  6  2  3  7  8  4
4  7  4  3  8  9  1  6  5  2
5  6  2  8  7  4  5  3  1  9
6  8  5  1  9  7  6  2  4  3
7  3  9  4  5  8  2  1  6  7
8  2  6  7  3  1  4  8  9  5


 time used :8.11 ms

   0  1  2  3  4  5  6  7  8
0  1  8  2  4  5  7  9  3  6
1  5  3  9  2  6  8  4  7  1
2  4  7  6  1  3  9  5  2  8
3  9  1  5  6  2  3  7  8  4
4  7  4  3  8  9  1  6  5  2
5  6  2  8  7  4  5  3  1  9
6  8  5  1  9  7  6  2  4  3
7  3  9  4  5  8  2  1  6  7
8  2  6  7  3  1  4  8  9  5


 time used :8.09 ms

   0  1  2  3  4  5  6  7  8
0  1  8  2  4  5  7  9  3  6
1  5  3  9  2  6  8  4  7  1
2  4  7  6  1  3  9  5  2  8
3  9  1  5  6  2  3  7  8  4
4  7  4  3  8  9  1  6  5  2
5  6  2  8  7  4  5  3  1  9
6  8  5  1  9  7  6  2  4  3
7  3  9  4  5  8  2  1  6  7
8  2  6  7  3  1  4  8  9  5


 time used :7.97 ms

   0  1  2  3  4  5  6  7  8
0  1  8  